In [1]:
import numpy as np
from theano import shared
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

# the pymc3 version used - 3.11.5
import pymc3 as pm
import matplotlib.pyplot as plt

DATA_DIR = '/final_train_test_split/'
plt.rcParams["font.family"] = "Times New Roman"
plt.style.use('seaborn-ticks')

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def hbm_vary_intercepts_slopes_model(train_df, test_df, selected_names, individual_names, feat_dim):
    """
    This the main function for hierarchical Bayesian model(HBM) used in the paper 
    <PREDICTING BATTERY LIFETIME UNDER VARYING USAGE CONDITIONS FROM EARLY AGING DATA>

    intput_params: 
        train_df - the training set with necessary group/individual features
        test_df - the test set including with necessary group/individual features
        selected_names - all feature column names
        individual_names - individual-level feature column names
        feat_dim - individual-level feature number, will influening the HBM model structure
    outputs:
        tol_rmse_hierarchical - average Root-mean-square-error (RMSE) for predictions on the test set
        tol_mape_hierarchical - average mean-absolute-percentage-error (MAPE) for predictions on the test set

    """

    # Using optimal 4 clusters setting mentioned in the paper
    group_num = len(train_df.cluster_c4.unique())
    with_group_names = selected_names.copy()
    with_group_names.append('cluster_c4')

    x_train, x_test = train_df.loc[:, with_group_names], test_df.loc[:, with_group_names]
    y_train, y_test = train_df.loc[:,'Lifetime'], test_df.loc[:,'Lifetime']

    # calculate group level feature for each cluster
    group_level_variables_train = np.zeros((group_num,1))
    for group_idx in range(group_num):
        mean_gf = np.mean(x_train.loc[x_train.cluster_c4 == group_idx, 'added_stress'].values)
        group_level_variables_train[group_idx,0] = mean_gf

    raw_group_ind = x_train.loc[:,'cluster_c4'].values
    raw_group_ind = raw_group_ind.astype(int)
    group_ind = shared(np.asarray(raw_group_ind))

    # Standard Scaler for both train and test set
    transformer = StandardScaler()
    processed_x_train = transformer.fit_transform(x_train.loc[:,individual_names])
    processed_x_test = transformer.transform(x_test.loc[:,individual_names])

    # Main HBM construction
    with pm.Model() as hierarchical_model:
        data = pm.Data('data', processed_x_train)
        group_data = pm.Data('group_data', group_level_variables_train)

        g_intercept = pm.Normal('g_intercept', mu = 12, sigma=10)
        g = pm.Normal('g', mu = 0, sigma = 1) # hyperprior intercept
        mu_intercept = g_intercept + g * group_data[:,0]
        intercept = pm.Deterministic('intercept', mu_intercept)

        # hyperprior slope0 - slope2
        g_slope0 = pm.Normal('g_slope0', mu = 0, sigma = 10, shape = 2) 
        mu_slope0 =  g_slope0[0] + g_slope0[1] * group_data[:,0]
        slope_0 = pm.Deterministic('slope_0', mu_slope0)

        g_slope1 = pm.Normal('g_slope1', mu = 0, sigma = 10, shape = 2) 
        mu_slope1 = g_slope1[0] + g_slope1[1] * group_data[:,0] 
        slope_1 = pm.Deterministic('slope_1', mu_slope1)

        # Two model construction choices: 2 or 3 individual-level features
        if feat_dim == 2:
            g_noise = pm.HalfCauchy('g_noise', beta=1)
            noise = pm.HalfCauchy('noise', beta=g_noise, shape=group_num)
            sds = noise[group_ind]
            y_est = intercept[group_ind] + slope_0[group_ind] * data[:,0] + slope_1[group_ind] * data[:,1]
        elif feat_dim == 3:
            g_slope2 = pm.Normal('g_slope2', mu = 0, sigma = 10, shape = 2) # hyperprior slope0 - slope2
            mu_slope2 = g_slope2[0] + g_slope2[1] * group_data[:,0]
            slope_2 = pm.Deterministic('slope_2', mu_slope2)
            g_noise = pm.HalfCauchy('g_noise', beta=1)
            noise = pm.HalfCauchy('noise', beta=g_noise, shape=group_num)
            sds = noise[group_ind]
            y_est = intercept[group_ind] + slope_0[group_ind] * data[:,0] + slope_1[group_ind] * data[:,1] + slope_2[group_ind] * data[:,2]
        else:
            raise ValueError('The individual level features number is either 2 or 3.')
        
        # sampling/inference step, the choice of tune and trace is set to 1000 which is common in analysis
        likelihood = pm.Normal('y_like', mu=y_est, sd=sds, observed=y_train.values)
        hierarchical_trace = pm.sample(1000, tune=1000, target_accept=0.99,return_inferencedata=False)

    with hierarchical_model:
        all_test = processed_x_test
        all_labels = y_test.values

        raw_group_ind = x_test.loc[:,'cluster_c4'].values
        raw_group_ind = raw_group_ind.astype(int)
        group_ind.set_value(np.asarray(raw_group_ind))
        
        pm.set_data({'data': all_test}, model=hierarchical_model)
        all_posterior_samples = pm.sample_posterior_predictive(hierarchical_trace)
        all_pred = all_posterior_samples["y_like"].mean(axis=0)
        tol_mape_hierarchical = mean_absolute_percentage_error(y_true = all_labels, 
                                            y_pred = all_pred)
        tol_rmse_hierarchical = mean_squared_error(y_true= all_labels, 
                                            y_pred = all_pred, squared=False)
                           
    return [tol_rmse_hierarchical, tol_mape_hierarchical, hierarchical_trace, all_pred, all_posterior_samples]

In [3]:
DATA_DIR = 'final_train_test_split/'
train_df = pd.read_csv(DATA_DIR + 'train.csv',index_col=0)
testin_df = pd.read_csv(DATA_DIR + 'test_in.csv',index_col=0)
testout_df = pd.read_csv(DATA_DIR + 'test_out.csv',index_col=0)

In [4]:
# Calculate the corresponding features mentioned in the paper
# avg-rate: Stress_avg - the group level feature
# log_mean_dqdv_dchg_mid_3_0, log_delta_CV_time_03 are the top two individual-level features mention in Table 1

train_df['avg_rate'] = (train_df.ChgC_rate + train_df.DchgC_rate) /2
testin_df['avg_rate'] = (testin_df.ChgC_rate + testin_df.DchgC_rate) /2
testout_df['avg_rate'] = (testout_df.ChgC_rate + testout_df.DchgC_rate) /2

train_df['log_mean_dqdv_dchg_mid_3_0'] = np.log(abs(train_df.mean_dqdv_dchg_mid_3_0))
testin_df['log_mean_dqdv_dchg_mid_3_0'] = np.log(abs(testin_df.mean_dqdv_dchg_mid_3_0))
testout_df['log_mean_dqdv_dchg_mid_3_0'] = np.log(abs(testout_df.mean_dqdv_dchg_mid_3_0))

train_df['log_delta_CV_time_03'] = np.log(abs(train_df.delta_CV_time_03))
testin_df['log_delta_CV_time_03'] = np.log(abs(testin_df.delta_CV_time_03))
testout_df['log_delta_CV_time_03'] = np.log(abs(testout_df.delta_CV_time_03))

train_df['log(Lifetime)'] = np.log(train_df.Lifetime)
testin_df['log(Lifetime)'] = np.log(testin_df.Lifetime)
testout_df['log(Lifetime)'] = np.log(testout_df.Lifetime)

In [5]:
# This the experiment on HBM (with 2 individual level features)
# Results recorded in Table 2
selected_names = ['added_stress', 'log_mean_dqdv_dchg_mid_3_0','log_delta_CV_time_03']
individual_names = [ 'log_mean_dqdv_dchg_mid_3_0','log_delta_CV_time_03']

rmse_hbm_train, mape_hbm_train, sample_trace_train, all_preds_train, posterior_hbm_train = hbm_vary_intercepts_slopes_model(train_df, train_df, selected_names, individual_names, feat_dim=2)
rmse_hbm_testin, mape_hbm_testin, sample_trace_testin, all_preds_testin, posterior_hbm_testin = hbm_vary_intercepts_slopes_model(train_df, testin_df, selected_names, individual_names, feat_dim=2)
rmse_hbm_testout, mape_hbm_testout, sample_trace_testout, all_preds_testout, posterior_hbm_testout  = hbm_vary_intercepts_slopes_model(train_df, testout_df, selected_names, individual_names, feat_dim=2)

print('train set: RMSE-{} MAPE-{}'.format(rmse_hbm_train.round(3), mape_hbm_train.round(3)))
print('testin set: RMSE-{} MAPE-{}'.format(rmse_hbm_testin.round(3), mape_hbm_testin.round(3)))
print('testout set: RMSE-{} MAPE-{}'.format(rmse_hbm_testout.round(3), mape_hbm_testout.round(3)))

# Save the whole posterior for prediction as csv files
# Use them to generate Fig 5 in the paper
np.savetxt('hbm_posterior_train.csv', posterior_hbm_train['y_like'])
np.savetxt('hbm_posterior_testin.csv', posterior_hbm_testin['y_like'])
np.savetxt('hbm_posterior_testout.csv', posterior_hbm_testout['y_like'])

# Save posterior for parameters as csv files
# Use them to generate Fig 5 in the paper
np.savetxt('hbm_intercepts.csv', sample_trace_train['intercept'])
np.savetxt('hbm_slope0s.csv', sample_trace_train['slope_0'])
np.savetxt('hbm_slope1s.csv', sample_trace_train['slope_1'])
np.savetxt('hbm_noises.csv', sample_trace_train['noise'])


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 81 seconds.


Could not locate executable g77
Could not locate executable f77
Could not locate executable ifort
Could not locate executable ifl
Could not locate executable f90
Could not locate executable DF
Could not locate executable efl


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 79 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 79 seconds.


train set: RMSE-3.261 MAPE-0.186
testin set: RMSE-3.054 MAPE-0.168
testout set: RMSE-7.337 MAPE-0.218


In [51]:
# This the experiment on HBM (with 3 individual level features)
# Results recorded in Table 2

selected_names = ['added_stress', 'log_mean_dqdv_dchg_mid_3_0','log_delta_CV_time_03','DoD']
individual_names = [ 'log_mean_dqdv_dchg_mid_3_0','log_delta_CV_time_03','DoD']

rmse_hbm_train, mape_hbm_train, _, _, _ = hbm_vary_intercepts_slopes_model(train_df, train_df, selected_names, individual_names, feat_dim=3)
rmse_hbm_testin, mape_hbm_testin, _, _, _ = hbm_vary_intercepts_slopes_model(train_df, testin_df, selected_names, individual_names, feat_dim=3)
rmse_hbm_testout, mape_hbm_testout, _, _, _ = hbm_vary_intercepts_slopes_model(train_df, testout_df, selected_names, individual_names, feat_dim=3)

print('train set: RMSE-{} MAPE-{}'.format(rmse_hbm_train.round(3), mape_hbm_train.round(3)))
print('testin set: RMSE-{} MAPE-{}'.format(rmse_hbm_testin.round(3), mape_hbm_testin.round(3)))
print('testout set: RMSE-{} MAPE-{}'.format(rmse_hbm_testout.round(3), mape_hbm_testout.round(3)))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope2, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 117 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope2, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 113 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope2, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 101 seconds.


train set: RMSE-3.103 MAPE-0.175
testin set: RMSE-2.832 MAPE-0.156
testout set: RMSE-7.551 MAPE-0.242
